In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
#preprocessing the images
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
# Converts a PIL Image or numpy array to a PyTorch tensor
# Normalize tensor values

In [ ]:
training_data=torchvision.datasets.CIFAR100(root = './data',
                                            train=True,
                                            download=True,
                                            transform=transform)
testing_data=torchvision.datasets.CIFAR100(root = './data',
                                            train=False,
                                            download=True,
                                            transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
batch_size=64

train_dataloader=torch.utils.data.DataLoader(training_data,batch_size=batch_size)
test_dataloader=torch.utils.data.DataLoader(testing_data,batch_size=batch_size)

In [ ]:
#Architecture

class CNN(nn.Module):

  def __init__(self):
    super(CNN,self).__init__()

    self.conv1 = nn.Conv2d(3,6,5)
    self.pool = nn.MaxPool2d(2,2)
    self.conv2 = nn.Conv2d(6,16,5)
    self.flatten = nn.Flatten(start_dim=1)
    self.fc1 = nn.Linear(1600,200)
    self.fc2 = nn.Linear(200,160)
    self.fc3 = nn.Linear(160,120)
    self.fc4 = nn.Linear(120,100)
  def forward(self,x):
    x = self.conv1(x) #i/p:3,32*32     o/p:6,28*28
    #x = F.relu(x)
    x = self.pool(x) # coverting 6,14*14
    #print(x.shape)
    x = self.conv2(x) #16*10*10
    #print(x.shape)
    x = self.flatten(x)
    #print(x.shape)
    x = self.fc1(x)
    #print(x.shape)
    x = F.relu(x)
    #print(x.shape)
    x = self.fc2(x)
    #print(x.shape)
    x = F.sigmoid(x)
    x = self.fc3(x)
    x = F.relu(x)
    x = self.fc4(x)
    #print(x.shape)
    return x

In [ ]:
device = ("cuda" if torch.cuda.is_available() else 'cpu')

In [ ]:
model=CNN()
model.to(device)
model

CNN(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=1600, out_features=200, bias=True)
  (fc2): Linear(in_features=200, out_features=160, bias=True)
  (fc3): Linear(in_features=160, out_features=120, bias=True)
  (fc4): Linear(in_features=120, out_features=100, bias=True)
)

In [ ]:
loss_fn=nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(model.parameters(),lr=1e-2)

In [ ]:
def training(train_dataloader,model,loss_,optimizer):
  model.train() # Set the model to training mode
#train_loss = 0.0
#pass every batch of data to the model
  for batch, (X,y) in enumerate(train_dataloader):
    X = X.to(device)
    y = y.to(device)

    pred = model(X) # Forward pass: Compute model predictions
    #print(pred)
    #print(y)
    loss = loss_fn(pred,y) # Compute the loss
    #print(loss)
    optimizer.zero_grad() # Zero the gradients
    loss.backward()  # Backpropagate the gradients
    optimizer.step() # Update the model's parameters

    if batch % 100 ==0:
      print(f'loss:{loss.item()}')

In [ ]:
def testing(test_dataloader,model,loss_):
  model.eval()  # Set the model to evaluation mode
  test_loss = 0.0 # Initialize the test loss
  correct = 0 # Initialize the number of correct predictions
  for X,y in test_dataloader:
    X = X.to(device)
    y = y.to(device)

    pred = model(X) # Forward pass: Compute model predictions
    #print(pred)
    correct += (pred.argmax(1) == y).type(torch.float).sum().item() # Calculate the number of correct predictions
    test_loss += loss_fn(pred,y).item()  # Compute the loss for this batch and accumulate it
  test_loss = test_loss/len(test_dataloader)
  print (len(test_dataloader.dataset))
  accuracy = (correct/len(test_dataloader.dataset))*100 # Calculate accuracy as the percentage of correct predictions
  print (f'Accuracy : {accuracy} with test error {test_loss}')
    #print(correct)

In [ ]:
epochs = 9 #how many times you have to send data to the model

for t in range(epochs):
  training(train_dataloader,model,loss_fn,optimizer) #Training phase: Pass training data through the model, compute loss, and optimize
  testing(test_dataloader,model,loss_fn) #Testing phase: Evaluate the model on the test data
  print(f' ==================== EPOCH : {t} ============================')

loss:4.601317405700684
loss:4.593739032745361
loss:4.630115985870361
loss:4.632089614868164
loss:4.606237411499023
loss:4.6066412925720215
loss:4.603342533111572
loss:4.600522994995117
10000
Accuracy : 1.0 with test error 4.606433999006915
 ==================== EPOCH : 0 ============================
loss:4.608252048492432
loss:4.600268840789795
loss:4.611692905426025
loss:4.6190314292907715
loss:4.606363773345947
loss:4.604320526123047
loss:4.604795932769775
loss:4.602349758148193
10000
Accuracy : 1.04 with test error 4.605513098892892
 ==================== EPOCH : 1 ============================
loss:4.608470439910889
loss:4.60114860534668
loss:4.60848331451416
loss:4.615307807922363
loss:4.605639934539795
loss:4.6034746170043945
loss:4.606010913848877
loss:4.60255241394043
10000
Accuracy : 1.16 with test error 4.604990327434176
 ==================== EPOCH : 2 ============================
loss:4.608027935028076
loss:4.600627899169922
loss:4.607518196105957
loss:4.613194465637207
loss:4